In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
k = 10; m = 25
x_data = np.concatenate([np.linspace(0, 0.3, k),      # small x values
                         np.linspace(0.6, 1, m-k)])   # large x values
x_data += np.random.rand(len(x_data))*0.05            # add small noise
y_data = [1]*k + [0]*(m-k)  # label 0 for small x, label 1 for large x 

In [3]:
def logistic(params, x):
    return 1/(1+np.exp(-(params[0]*x + params[1])))

def log_lik_grad(params, x_data):
    grad_a_part1 = x_data[:k]
    grad_a_part2 = -x_data*logistic(params, x_data)
    grad_b_part1 = np.ones(k)
    grad_b_part2 = -logistic(params, x_data)
    return np.array([grad_a_part1.sum() + grad_a_part2.sum(),
                     grad_b_part1.sum() + grad_b_part2.sum()])

In [4]:
def make_plot(epoch, x_data, y_data, params, params_hist, log_lik_hist):
    pred_x = np.linspace(0,1,100)
    pred_y = logistic(params, pred_x)
    plt.subplots(2, 2, figsize=(8,6))
    plt.subplot(2, 2, 1)
    plt.scatter(x_data, y_data, s=20, facecolors='none', edgecolors='k')
    plt.plot(pred_x, pred_y)
    plt.title('data and fit', fontsize=20)
    plt.subplot(2, 2, 2)
    plt.plot(log_lik_hist, color='r')
    plt.title('log likelihood', fontsize=20)
    plt.xlabel('epoch', fontsize=15)
    plt.subplot(2, 2, 3)
    plt.plot(params_hist[:, 0])
    plt.title('a', fontsize=20)
    plt.xlabel('epoch', fontsize=15)
    plt.subplot(2, 2, 4)
    plt.plot(params_hist[:, 1])
    plt.title('b', fontsize=20)
    plt.xlabel('epoch', fontsize=15)
    print(f'epoch {epoch:d} a={params[0]:0.4f} b={params[1]:0.4f}'
          f' a*0.2+b={params[0]*0.2+params[1]:0.4f}',
          f' a*0.8+b={params[0]*0.8+params[1]:0.4f}')
    plt.tight_layout()
    plt.savefig(f'/tmp/log_reg_viz_epoch{epoch:d}.jpg', dpi=150)
    plt.close()

In [5]:
def log_lik(params, x_data, k):
    a, b = params
    return np.sum(a*x_data[:k] + b) - np.sum(np.log(1 + np.exp(a*x_data + b)))

params = np.zeros(2)
lr = 0.1
params_hist = []
log_lik_hist = []
for epoch in range(100000):  # grad ascent
    grad = log_lik_grad(params, x_data)
    params += lr*grad
    params_hist.append(np.copy(params))
    log_lik_val = log_lik(params, x_data, k)
    log_lik_hist.append(log_lik_val)
    if (epoch < 20 or (epoch < 100 and epoch % 10 == 0) 
        or (epoch < 200 and epoch % 20 == 0) or (epoch % 1000 == 0)):
        make_plot(epoch, x_data, y_data, params,
                  np.array(params_hist), np.array(log_lik_hist))

epoch 0 a=-0.5320 b=-0.2500 a*0.2+b=-0.3564  a*0.8+b=-0.6756
epoch 1 a=-0.8397 b=-0.1664 a*0.2+b=-0.3343  a*0.8+b=-0.8381
epoch 2 a=-1.1013 b=-0.0352 a*0.2+b=-0.2554  a*0.8+b=-0.9162
epoch 3 a=-1.3437 b=0.1007 a*0.2+b=-0.1681  a*0.8+b=-0.9743
epoch 4 a=-1.5733 b=0.2320 a*0.2+b=-0.0826  a*0.8+b=-1.0266
epoch 5 a=-1.7921 b=0.3569 a*0.2+b=-0.0015  a*0.8+b=-1.0768
epoch 6 a=-2.0011 b=0.4753 a*0.2+b=0.0750  a*0.8+b=-1.1256
epoch 7 a=-2.2010 b=0.5875 a*0.2+b=0.1473  a*0.8+b=-1.1733
epoch 8 a=-2.3923 b=0.6939 a*0.2+b=0.2155  a*0.8+b=-1.2199
epoch 9 a=-2.5755 b=0.7951 a*0.2+b=0.2800  a*0.8+b=-1.2653
epoch 10 a=-2.7512 b=0.8915 a*0.2+b=0.3413  a*0.8+b=-1.3095
epoch 11 a=-2.9199 b=0.9834 a*0.2+b=0.3994  a*0.8+b=-1.3525
epoch 12 a=-3.0820 b=1.0712 a*0.2+b=0.4548  a*0.8+b=-1.3944
epoch 13 a=-3.2379 b=1.1552 a*0.2+b=0.5076  a*0.8+b=-1.4352
epoch 14 a=-3.3881 b=1.2356 a*0.2+b=0.5580  a*0.8+b=-1.4748
epoch 15 a=-3.5328 b=1.3128 a*0.2+b=0.6063  a*0.8+b=-1.5134
epoch 16 a=-3.6725 b=1.3870 a*0.2+b=0.652

In [7]:
import sys
!{sys.executable} -m pip install torch

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [5]:
def torch_log_lik(params, x_data, k):
    a, b = params
    return torch.sum(a*x_data[:k] + b) - torch.sum(torch.log(1 + torch.exp(a*x_data + b)))

import torch
params_tensor = torch.zeros(2, requires_grad=True)
optimizer = torch.optim.SGD([params_tensor], lr=0.1)
params_hist = []
log_lik_hist = []
x_data_tensor = torch.from_numpy(x_data)
for epoch in range(100000):  # grad ascent
    optimizer.zero_grad()
    loss = -torch_log_lik(params_tensor, x_data_tensor, k).sum()
    loss.backward()
    optimizer.step()
    params_hist.append(np.copy(params_tensor.detach().numpy()))
    log_lik_hist.append(-loss.item())
    if (epoch < 20 or (epoch < 100 and epoch % 10 == 0) 
        or (epoch < 500 and epoch % 50 == 0) or (epoch % 1000 == 0)):
        make_plot(epoch, x_data, y_data, params_tensor.detach().numpy(),
                  np.array(params_hist), np.array(log_lik_hist))

epoch 0 a=-0.5321 b=-0.2500 a*0.2+b=-0.3564  a*0.8+b=-0.6757
epoch 1 a=-0.8399 b=-0.1664 a*0.2+b=-0.3344  a*0.8+b=-0.8383
epoch 2 a=-1.1016 b=-0.0352 a*0.2+b=-0.2556  a*0.8+b=-0.9166
epoch 3 a=-1.3442 b=0.1007 a*0.2+b=-0.1682  a*0.8+b=-0.9747
epoch 4 a=-1.5739 b=0.2320 a*0.2+b=-0.0828  a*0.8+b=-1.0271
epoch 5 a=-1.7928 b=0.3569 a*0.2+b=-0.0016  a*0.8+b=-1.0773
epoch 6 a=-2.0019 b=0.4753 a*0.2+b=0.0750  a*0.8+b=-1.1262
epoch 7 a=-2.2019 b=0.5876 a*0.2+b=0.1472  a*0.8+b=-1.1739
epoch 8 a=-2.3933 b=0.6941 a*0.2+b=0.2155  a*0.8+b=-1.2205
epoch 9 a=-2.5766 b=0.7954 a*0.2+b=0.2801  a*0.8+b=-1.2659
epoch 10 a=-2.7524 b=0.8919 a*0.2+b=0.3414  a*0.8+b=-1.3101
epoch 11 a=-2.9212 b=0.9839 a*0.2+b=0.3996  a*0.8+b=-1.3531
epoch 12 a=-3.0834 b=1.0717 a*0.2+b=0.4551  a*0.8+b=-1.3950
epoch 13 a=-3.2395 b=1.1558 a*0.2+b=0.5079  a*0.8+b=-1.4357
epoch 14 a=-3.3897 b=1.2364 a*0.2+b=0.5585  a*0.8+b=-1.4754
epoch 15 a=-3.5346 b=1.3137 a*0.2+b=0.6068  a*0.8+b=-1.5139
epoch 16 a=-3.6744 b=1.3881 a*0.2+b=0.653